# SQL Alchemy: Part 1

| Key              | Value                                                                                                                                                                                                                                                                                                                                                          |
|:-----------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Course Codes** | DAT 2201, DAT 3103, BBT 3104, MCS 8104, MIT 8107, BBT 4106                                                                                                                                                                                                                                                                                                     |
| **Course Names** | DAT 2201: Database Design and SQL (Week 1-3 of 13), <br/>DAT 3103: Principles of Data Engineering (Week 1-3 of 13), <br/>BBT 3104: Advanced Database Systems (Week 7-9 of 13), <br/>MCS 8104: Database Management Systems (Week 1-3 of 13), <br/>MIT 8107: Advanced Database Systems (Week 1-3 of 13), <br/>BBT 4106: Business Intelligence I (Week 4-6 of 13) |
| **Semester**     | May to July 2026                                                                                                                                                                                                                                                                                                                                               |
| **Lecturer**     | Allan Omondi                                                                                                                                                                                                                                                                                                                                                   |
| **Contact**      | aomondi@strathmore.edu                                                                                                                                                                                                                                                                                                                                         |
| **Note**         | The lecture contains both theory and practice.<br/>This notebook forms part of the practice.<br/>It is intended for educational purpose only.<br/>Recommended citation: [BibTex](https://raw.githubusercontent.com/course-files/ObjectRelationalMapping/refs/heads/main/RecommendedCitation.bib)                                                               |


## Overall Architecture

- The overall architecture is as follows:

**Frontend → Reverse Proxy → API (e.g., Flask or FastAPI) → Backend (e.g., a server running Python) → SQLAlchemy Core (Reads) and SQLAlchemy ORM (Writes) → Database**

- According to the [SQLAlchemy documentation (2025)](https://docs.sqlalchemy.org):
  - **SQLAlchemy Core** is the foundational architecture for SQLAlchemy as a “database toolkit.” The library provides tools for managing connectivity to a database, interacting with database queries and results, and programmatic construction of SQL statements.
  - **SQLAlchemy ORM** builds upon SQLAlchemy Core to provide optional Object Relational Mapping (ORM) capabilities. This includes an additional configuration layer allowing user-defined Python classes to be mapped to database tables and other constructs, as well as an object persistence mechanism known as the Session. It then extends the Core-level SQL Expression Language to allow SQL queries to be composed and invoked in terms of user-defined objects.

## Install SQLAlchemy

- We need to install SQLAlchemy to work with databases in Python.

In [1]:
!pip install sqlalchemy

## Import SQLAlchemy

- We then import the SQLAlchemy library into the Python environment.

In [2]:
import sqlalchemy

The library contains several packages. An alternative way of importing the library is to import specific packages as needed:

In [3]:
from sqlalchemy import exc

Another alternative is to import all packages from SQLAlchemy:

In [4]:
from sqlalchemy import *

## Check the Version of SQLAlchemy

- We can confirm the version of SQLAlchemy installed using the `__version__` attribute. This notebook is based on version `2.0.44` of SQLAlchemy.

In [5]:
sqlalchemy.__version__

'2.0.44'

## Create an Engine to Connect to the Database

- `create_engine()` is a function in SQLAlchemy that creates a new SQLAlchemy Engine instance. An Engine is the starting point for any SQLAlchemy application that wants to interact with a database. It manages the connection pool and provides a source of database connections.
- `echo=True` instructs SQLAlchemy to log all the SQL statements it executes to the console. This is useful for debugging and understanding what SQLAlchemy is doing for you behind the scenes.

In [6]:
engine = create_engine('sqlite+pysqlite:///mydatabase.db', echo=True)

## Connect to the Database using DataGrip

- Connect to the SQLite database using DataGrip (or any other similar software) to verify that the database file `mydatabase.db` has been created.
- In DataGrip, create a new data source and select SQLite. Then, select the `mydatabase.db` file to connect to the database.
- `mydatabase.db` should have been created in the current working directory when you executed the `create_engine()` function.

![img.png](assets/images/1_DataGripTOSQLLite.png)

## Connect to the Database using the engine in Python

In [7]:
conn = engine.connect()

## Execute Raw SQL Statements using SQLAlchemy Core

- We can also execute raw SQL statements using the connection object.
- Each SQL statement should be wrapped in the `text()` function available in SQLAlchemy.
- The `commit()` method then commits any changes to the database.

In [8]:
conn.execute(text('CREATE TABLE IF NOT EXISTS supplier (name VARCHAR(255), rating int)'))

conn.commit()

2025-12-10 15:27:00,555 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-10 15:27:00,556 INFO sqlalchemy.engine.Engine CREATE TABLE IF NOT EXISTS supplier (name VARCHAR(255), rating int)
2025-12-10 15:27:00,557 INFO sqlalchemy.engine.Engine [generated in 0.00211s] ()
2025-12-10 15:27:00,559 INFO sqlalchemy.engine.Engine COMMIT


- Confirm that the `supplier` table has been created in the `mydatabase.db` database using DataGrip or any other similar software.

![img.png](assets/images/2_CreateTableConfirmation.png)

In [9]:
conn.execute(text('INSERT INTO supplier (name, rating) VALUES ("Ruiru II Farm", 3);'))

conn.commit()

2025-12-10 15:27:00,579 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-10 15:27:00,580 INFO sqlalchemy.engine.Engine INSERT INTO supplier (name, rating) VALUES ("Ruiru II Farm", 3);
2025-12-10 15:27:00,581 INFO sqlalchemy.engine.Engine [generated in 0.00210s] ()
2025-12-10 15:27:00,583 INFO sqlalchemy.engine.Engine COMMIT


- Confirm that the new record has been inserted.

![img.png](assets/images/3_Insert_Confirmation.png)

- It is recommended to limit the use of the connection object to a specific context. This can be done using a `with` statement, for example:

In [10]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM supplier"))
    print(result.all())

2025-12-10 15:27:00,605 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-10 15:27:00,607 INFO sqlalchemy.engine.Engine SELECT * FROM supplier
2025-12-10 15:27:00,608 INFO sqlalchemy.engine.Engine [generated in 0.00267s] ()
[('Ruiru II Farm', 3), ('Narok 41 Ranch', 0.5), ('Ngong Green House', 2), ('Makueni Fruits', 4.3), ('Limuru Gardens', 4.1), ('Naivasha Florals', 3.4), ('Ruiru II Farm', 3)]
2025-12-10 15:27:00,610 INFO sqlalchemy.engine.Engine ROLLBACK


### Create
- Textual SQL is not the usual way we work with SQLAlchemy. However, when using textual SQL, a Python literal value, even non-strings like integers or dates, **should NEVER be 'stringified' directly into an SQL string**; a parameter should always be used. This avoids SQL injection attacks.

In [11]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO supplier (name, rating) VALUES (:name, :rating)"),
        [{"name": "Narok 41 Ranch", "rating": 4.8}, {"name": "Ngong Green House", "rating": 2}],
    )
    conn.commit()

2025-12-10 15:27:00,629 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-10 15:27:00,630 INFO sqlalchemy.engine.Engine INSERT INTO supplier (name, rating) VALUES (?, ?)
2025-12-10 15:27:00,631 INFO sqlalchemy.engine.Engine [generated in 0.00210s] [('Narok 41 Ranch', 4.8), ('Ngong Green House', 2)]
2025-12-10 15:27:00,633 INFO sqlalchemy.engine.Engine COMMIT


- The `BEGIN (implicit)` in the SQLAlchemy Core console means that SQLAlchemy has automatically started a new database transaction for you, without you explicitly calling `begin()`.

- This usually happens when you execute a statement that modifies the database (like `INSERT`, `UPDATE`, or `DELETE`) and no transaction is currently active. The transaction will remain open until you explicitly call `commit()` or `rollback()`.

- Alternatively, we can use the `begin()` method to handle transactions.

  - `engine.begin()` creates a connection and starts a transaction automatically. The transaction is committed if no exceptions occur, or rolled back if an exception is raised. This is useful for ensuring atomic operations and cleaner transaction management.
  - `engine.connect()` creates a database connection that you can use to execute SQL statements. You must manually manage transactions (i.e., call `commit()` or `rollback()` as needed).

- Using `engine.begin()` is generally preferred for transaction management, as it reduces the risk of leaving transactions open unintentionally. However, if you need more control over the connection and transaction lifecycle, you can choose to use `engine.connect()`. This gives you access to the rollback() method if needed.

- Analogy: Driving a car with an automatic transmission (using `begin()`) vs. a manual transmission (using `connect()`).

![img.png](assets/images/4_Automatic-vs-Manual-Transmission.png)

In [12]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO supplier (name, rating) VALUES (:name, :rating)"),
        [{"name": "Makueni Fruits", "rating": 4}, {"name": "Pwani Farm", "rating": 1.8}],
    )
    conn.commit()

2025-12-10 15:27:00,652 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-10 15:27:00,654 INFO sqlalchemy.engine.Engine INSERT INTO supplier (name, rating) VALUES (?, ?)
2025-12-10 15:27:00,655 INFO sqlalchemy.engine.Engine [cached since 0.02558s ago] [('Makueni Fruits', 4), ('Pwani Farm', 1.8)]
2025-12-10 15:27:00,658 INFO sqlalchemy.engine.Engine COMMIT


### Read

**Option 1:** Default print output

In [13]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT name, rating FROM supplier"))
    for row in result.fetchall():
        print(row)
    conn.commit()

2025-12-10 15:27:00,679 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-10 15:27:00,680 INFO sqlalchemy.engine.Engine SELECT name, rating FROM supplier
2025-12-10 15:27:00,681 INFO sqlalchemy.engine.Engine [generated in 0.00216s] ()
('Ruiru II Farm', 3)
('Narok 41 Ranch', 0.5)
('Ngong Green House', 2)
('Makueni Fruits', 4.3)
('Limuru Gardens', 4.1)
('Naivasha Florals', 3.4)
('Ruiru II Farm', 3)
('Narok 41 Ranch', 4.8)
('Ngong Green House', 2)
('Makueni Fruits', 4)
('Pwani Farm', 1.8)
2025-12-10 15:27:00,683 INFO sqlalchemy.engine.Engine COMMIT


**Option 2:** Customized print output

In [14]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT name, rating FROM supplier"))
    for row in result.fetchall():
        print(f"name: {row.name},  rating: {row.rating}")
    conn.commit()

2025-12-10 15:27:00,710 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-10 15:27:00,712 INFO sqlalchemy.engine.Engine SELECT name, rating FROM supplier
2025-12-10 15:27:00,713 INFO sqlalchemy.engine.Engine [cached since 0.03418s ago] ()
name: Ruiru II Farm,  rating: 3
name: Narok 41 Ranch,  rating: 0.5
name: Ngong Green House,  rating: 2
name: Makueni Fruits,  rating: 4.3
name: Limuru Gardens,  rating: 4.1
name: Naivasha Florals,  rating: 3.4
name: Ruiru II Farm,  rating: 3
name: Narok 41 Ranch,  rating: 4.8
name: Ngong Green House,  rating: 2
name: Makueni Fruits,  rating: 4
name: Pwani Farm,  rating: 1.8
2025-12-10 15:27:00,716 INFO sqlalchemy.engine.Engine COMMIT


**Option 3:** Print output and saved as a data frame

In [15]:
!pip install pandas

In [16]:
import pandas as pd

with engine.connect() as conn:
    result = conn.execute(text("SELECT name, rating FROM supplier"))
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    conn.commit()

display(df)

2025-12-10 15:27:02,881 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-10 15:27:02,882 INFO sqlalchemy.engine.Engine SELECT name, rating FROM supplier
2025-12-10 15:27:02,883 INFO sqlalchemy.engine.Engine [cached since 2.204s ago] ()
2025-12-10 15:27:02,885 INFO sqlalchemy.engine.Engine COMMIT


,name,rating
0,Ruiru II Farm,3.0
1,Narok 41 Ranch,0.5
2,Ngong Green House,2.0
3,Makueni Fruits,4.3
4,Limuru Gardens,4.1
5,Naivasha Florals,3.4
6,Ruiru II Farm,3.0
7,Narok 41 Ranch,4.8
8,Ngong Green House,2.0
9,Makueni Fruits,4.0


### Update

In [17]:
with engine.connect() as conn:
    conn.execute(
        text("UPDATE supplier SET rating = :rating WHERE name = :name"),
        {"name": "Makueni Fruits", "rating": 5}
    )
    conn.commit()

2025-12-10 15:27:03,037 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-10 15:27:03,038 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-10 15:27:03,040 INFO sqlalchemy.engine.Engine [generated in 0.00290s] (5, 'Makueni Fruits')
2025-12-10 15:27:03,042 INFO sqlalchemy.engine.Engine COMMIT


With multiple statements in one database transaction.

In [18]:
with engine.connect() as conn:
    conn.execute(
        text("UPDATE supplier SET rating = :rating WHERE name = :name"),
        {"name": "Makueni Fruits", "rating": 4.5}
    )
    conn.execute(
        text("UPDATE supplier SET rating = :rating WHERE name = :name"),
        {"name": "Narok 41 Ranch", "rating": 4.9}
    )
    result = conn.execute(
        text("SELECT name, rating FROM supplier WHERE name = :name"),
        {"name": "Makueni Fruits"}
    )
    for row in result.fetchall():
        print(row)
    conn.commit()

2025-12-10 15:27:03,097 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-10 15:27:03,099 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-10 15:27:03,100 INFO sqlalchemy.engine.Engine [cached since 0.06292s ago] (4.5, 'Makueni Fruits')
2025-12-10 15:27:03,104 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-10 15:27:03,105 INFO sqlalchemy.engine.Engine [cached since 0.06844s ago] (4.9, 'Narok 41 Ranch')
2025-12-10 15:27:03,107 INFO sqlalchemy.engine.Engine SELECT name, rating FROM supplier WHERE name = ?
2025-12-10 15:27:03,107 INFO sqlalchemy.engine.Engine [generated in 0.00090s] ('Makueni Fruits',)
('Makueni Fruits', 4.5)
('Makueni Fruits', 4.5)
2025-12-10 15:27:03,109 INFO sqlalchemy.engine.Engine COMMIT


### Delete

In [19]:
with engine.connect() as conn:
    conn.execute(
        text("DELETE FROM supplier WHERE name = :name"),
        {"name": "Pwani Farm"}
    )

    conn.commit()

2025-12-10 15:27:03,137 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-10 15:27:03,139 INFO sqlalchemy.engine.Engine DELETE FROM supplier WHERE name = ?
2025-12-10 15:27:03,140 INFO sqlalchemy.engine.Engine [generated in 0.00261s] ('Pwani Farm',)
2025-12-10 15:27:03,144 INFO sqlalchemy.engine.Engine COMMIT


### Rollback to a Savepoint
- A savepoint allows you to set a point within a transaction that you can roll back to without affecting the entire transaction. This is useful when you want to undo certain operations while keeping others intact.

In [20]:
with engine.connect() as conn:
    trans = conn.begin()  # start transaction
    try:
        conn.execute(
            text("UPDATE supplier SET rating = :rating WHERE name = :name"),
            {"name": "Makueni Fruits", "rating": 4.3}
        )
        # Create a savepoint
        savepoint_1 = conn.begin_nested()
        conn.execute(
            text("UPDATE supplier SET rating = :rating WHERE name = :name"),
            {"name": "Narok 41 Ranch", "rating": 1.2}
        )
        # Rollback to savepoint (undo last update only)
        savepoint_1.rollback()
        result = conn.execute(
            text("SELECT name, rating FROM supplier WHERE name = :name"),
            {"name": "Makueni Fruits"}
        )
        for row in result.fetchall():
            print(row)
        trans.commit()
    except sqlalchemy.exc.SQLAlchemyError:
        trans.rollback()
        print("Transaction rolled back")

2025-12-10 15:27:03,170 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-10 15:27:03,171 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-10 15:27:03,173 INFO sqlalchemy.engine.Engine [cached since 0.136s ago] (4.3, 'Makueni Fruits')
2025-12-10 15:27:03,175 INFO sqlalchemy.engine.Engine SAVEPOINT sa_savepoint_1
2025-12-10 15:27:03,177 INFO sqlalchemy.engine.Engine [no key 0.00127s] ()
2025-12-10 15:27:03,179 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-10 15:27:03,180 INFO sqlalchemy.engine.Engine [cached since 0.1435s ago] (1.2, 'Narok 41 Ranch')
2025-12-10 15:27:03,181 INFO sqlalchemy.engine.Engine ROLLBACK TO SAVEPOINT sa_savepoint_1
2025-12-10 15:27:03,182 INFO sqlalchemy.engine.Engine [no key 0.00062s] ()
2025-12-10 15:27:03,183 INFO sqlalchemy.engine.Engine SELECT name, rating FROM supplier WHERE name = ?
2025-12-10 15:27:03,185 INFO sqlalchemy.engine.Engine [cached since 0.078s ago] ('Makueni Fruits'

In [21]:
with engine.connect() as conn:
    trans = conn.begin()
    try:
        conn.execute(
            text("UPDATE supplier SET rating = :rating WHERE name = :name"),
            {"name": "Narok 41 Ranch", "rating": 0.5}
        )

        sp1 = conn.begin_nested()  # Savepoint 1
        conn.execute(
            text("UPDATE supplier SET rating = :rating WHERE name = :name"),
            {"name": "Ngong Green House", "rating": 0.9}
        )

        sp2 = conn.begin_nested()  # Savepoint 2
        conn.execute(
            text("UPDATE supplier SET rating = :rating WHERE name = :name"),
            {"name": "Narok 41 Ranch", "rating": 5.0}
        )
        sp2.rollback()  # Undo the last update only
        sp1.rollback()  # Undo up to Savepoint 1 (including changes after sp1)
        trans.commit()
    except sqlalchemy.exc.SQLAlchemyError:
        trans.rollback()

2025-12-10 15:27:03,213 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-10 15:27:03,215 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-10 15:27:03,216 INFO sqlalchemy.engine.Engine [cached since 0.1794s ago] (0.5, 'Narok 41 Ranch')
2025-12-10 15:27:03,219 INFO sqlalchemy.engine.Engine SAVEPOINT sa_savepoint_1
2025-12-10 15:27:03,221 INFO sqlalchemy.engine.Engine [no key 0.00155s] ()
2025-12-10 15:27:03,222 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-10 15:27:03,224 INFO sqlalchemy.engine.Engine [cached since 0.187s ago] (0.9, 'Ngong Green House')
2025-12-10 15:27:03,225 INFO sqlalchemy.engine.Engine SAVEPOINT sa_savepoint_2
2025-12-10 15:27:03,227 INFO sqlalchemy.engine.Engine [no key 0.00211s] ()
2025-12-10 15:27:03,228 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-10 15:27:03,229 INFO sqlalchemy.engine.Engine [cached since 0.1924s ago] (5.0, 'Narok 41 Ranch')
2025

## Close the Connection
- It is good practice to close the connection when you are done. This also commits anything that has not yet been committed up to this point.

In [22]:
conn.close()

## References
SQLAlchemy Project. (2025, December 5). _SQLAlchemy 2.0 Documentation._ SQLAlchemy. https://docs.sqlalchemy.org/en/20/
